### como temos poucos dados testes, por termos poucos dados no geral, os accuracy oscilam muito

### modelos


- ~~Logistic Regression (non-regularized)~~

- ~~Logistic Regression (regularized)~~

    - ~~using ridge~~

    - ~~using lasso~~


- ~~Redes Neuronais (não regularizada)~~

- ~~Redes Neuronais (regularizada)~~

- ~~Support Vector Classification (SVC) (nao regularizado)~~

- ~~Support Vector Classification (SVC) (regularizado)~~

In [808]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import datasets
from sklearn.svm import SVC

In [809]:
data = pd.read_csv('02balanced.csv')
data.sample(5)

,Name,Date,Sex_F,Appetite,SleepInd,StSpirit,pEffort,uaI,uaI(-1),DeltaWt (%),DeltaHrtwo,FatigInd
50,F_m,2020-05-04,0,3.0,-4.5,5.0,4.0,2.00,1.00,0.000000,17.0,Low/Minimal
72,J_f,2020-02-16,1,6.0,-6.0,5.0,1.0,0.00,2.11,0.000000,-6.0,Low/Minimal
314,F_f,2020-02-12,1,6.0,-3.5,3.0,8.0,2.37,1.81,-0.003384,-12.0,Risk/Caution
284,A_m,2020-01-23,0,4.0,-5.0,4.0,8.0,1.88,2.86,0.000000,0.0,Risk/Caution
353,J_f,2019-11-05,1,6.0,-4.5,6.0,5.0,2.00,2.24,-0.012048,0.0,Risk/Caution


In [810]:
def standardize_numeric_columns(df):
    # Separate numeric and non-numeric columns
    numeric_cols = ['Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                    'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']

    # Initialize StandardScaler
    scaler = StandardScaler()

    # Standardize only the numeric columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Display the resulting DataFrame
    return df 

df = standardize_numeric_columns(data)

In [811]:
def logistical_regression(df, sex="all", only_accuracy=False):
       """regressao logistica apresentada não está regularizada"""
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
       # Initialize the model
       model = LogisticRegression()

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)
       print("Accuracy")
       print(accuracy_score(y_test, y_pred))


       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       """
       metrics = {
              "Logistic Regression": {
                     "Accuracy": accuracy_score(y_test, y_pred),
                     "F1 Score": f1_score(y_test, y_pred, average='weighted'),
                     "Precision": precision_score(y_test, y_pred, average='weighted'),
                     "Recall": recall_score(y_test, y_pred, average='weighted')
              }}
       print(metrics)
       """

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)

logistical_regression(df, sex="all", only_accuracy=True)
logistical_regression(df, sex="F", only_accuracy=True)
logistical_regression(df, sex="M", only_accuracy=True)

Accuracy
0.6481481481481481
Confusion Matrix:
[[17  3  1]
 [ 3  5  7]
 [ 1  4 13]]
Accuracy
0.4444444444444444
Confusion Matrix:
[[4 1 3]
 [2 3 2]
 [3 4 5]]
Accuracy
0.4444444444444444
Confusion Matrix:
[[5 4 0]
 [2 3 6]
 [0 3 4]]


In [812]:
def logistical_regression_comPenalidade_RIDGE(df, sex="all", only_accuracy=False):
       """regressao logistica apresentada está regularizada:
       using Ridge (L2) regularization
       C parameter controls the strength of the regularization (with smaller values indicating stronger regularization
       # C is the inverse of regularization strength"""
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
       # Initialize the model
       model = LogisticRegression(penalty='l2', C=0.1)

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)

       # Calculate accuracy
       accuracy = accuracy_score(y_test, y_pred)
       print(f'Accuracy: {accuracy}')
       #accuracy = accuracy_score(y_train, model.predict(X_train))
       #print(f'Accuracy: {accuracy}')

       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)

logistical_regression_comPenalidade_RIDGE(df, sex="all", only_accuracy=True)
logistical_regression_comPenalidade_RIDGE(df, sex="F", only_accuracy=True)
logistical_regression_comPenalidade_RIDGE(df, sex="M", only_accuracy=True)

Accuracy: 0.6851851851851852
Confusion Matrix:
[[14  2  2]
 [ 3 10  7]
 [ 1  2 13]]
Accuracy: 0.7037037037037037
Confusion Matrix:
[[ 5  2  0]
 [ 3  4  1]
 [ 0  2 10]]
Accuracy: 0.5185185185185185
Confusion Matrix:
[[6 5 0]
 [2 3 3]
 [0 3 5]]


In [813]:
def logistical_regression_comPenalidade_LASSO(df, sex="all", only_accuracy=False):
       """regressao logistica apresentada está regularizada:
       using Lasso (L1) regularization
       C parameter controls the strength of the regularization (with smaller values indicating stronger regularization
       # C is the inverse of regularization strength"""
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
       # Initialize the model
       model = LogisticRegression(penalty='l1', C=0.1)

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)

       # Calculate accuracy
       accuracy = accuracy_score(y_test, y_pred)
       print(f'Accuracy: {accuracy}')
       #accuracy = accuracy_score(y_train, model.predict(X_train))
       #print(f'Accuracy: {accuracy}')

       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)

logistical_regression_comPenalidade_RIDGE(df, sex="all", only_accuracy=True)
logistical_regression_comPenalidade_RIDGE(df, sex="F", only_accuracy=True)
logistical_regression_comPenalidade_RIDGE(df, sex="M", only_accuracy=True)

Accuracy: 0.6296296296296297
Confusion Matrix:
[[14  6  1]
 [ 3  8  8]
 [ 0  2 12]]
Accuracy: 0.4444444444444444
Confusion Matrix:
[[3 2 1]
 [4 3 3]
 [2 3 6]]
Accuracy: 0.7037037037037037
Confusion Matrix:
[[9 1 0]
 [2 4 4]
 [0 1 6]]


In [814]:
def neural_network(df, sex="all"):
        """rede neural apresentada não está regularizada"""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


        # Neural Network Model
        nn = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=1000)
        nn.fit(X_train, y_train)
        nn_pred = nn.predict(X_test)
        nn_probs = nn.predict_proba(X_test)  # Probabilities for ROC-AUC

        metrics = {
                "Accuracy": accuracy_score(y_test, nn_pred),
                "F1 Score": f1_score(y_test, nn_pred, average='weighted'),
                "Precision": precision_score(y_test, nn_pred, average='weighted'),
                "Recall": recall_score(y_test, nn_pred, average='weighted'),
                "ROC-AUC": roc_auc_score(y_test, nn_probs, multi_class='ovr', average='weighted')
        }

        # Display metrics
        print(metrics)
        conf_matrix = confusion_matrix(y_test, nn_pred)
        print("Confusion Matrix:")
        print(conf_matrix)

neural_network(df, sex="all")
neural_network(df, sex="F")
neural_network(df, sex="M")

{'Accuracy': 0.6338028169014085, 'F1 Score': np.float64(0.6328216030581164), 'Precision': np.float64(0.6322653268539109), 'Recall': np.float64(0.6338028169014085), 'ROC-AUC': np.float64(0.7597652982401372)}
Confusion Matrix:
[[12  7  1]
 [ 6 13  6]
 [ 1  5 20]]
{'Accuracy': 0.5833333333333334, 'F1 Score': np.float64(0.5861792528459195), 'Precision': np.float64(0.613095238095238), 'Recall': np.float64(0.5833333333333334), 'ROC-AUC': np.float64(0.7033427568558286)}
Confusion Matrix:
[[ 6  2  1]
 [ 5 10  4]
 [ 1  2  5]]
{'Accuracy': 0.4722222222222222, 'F1 Score': np.float64(0.47519335974344457), 'Precision': np.float64(0.48520923520923515), 'Recall': np.float64(0.4722222222222222), 'ROC-AUC': np.float64(0.6996198830409357)}
Confusion Matrix:
[[8 3 0]
 [3 6 8]
 [0 5 3]]


In [815]:
def neural_network_regularizada(df, sex="all"):
        """rede neural apresentada está regularizada
        Ridge regularization) is applied through the alpha parameter."""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


        # Neural Network Model
        nn = MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=250, alpha=0.01) #L2 regularization
        #(also known as Ridge regularization) is applied through the alpha parameter.
        nn.fit(X_train, y_train)
        nn_pred = nn.predict(X_test)
        nn_probs = nn.predict_proba(X_test)  # Probabilities for ROC-AUC

        metrics = {
                "Accuracy": accuracy_score(y_test, nn_pred),
                "F1 Score": f1_score(y_test, nn_pred, average='weighted'),
                "Precision": precision_score(y_test, nn_pred, average='weighted'),
                "Recall": recall_score(y_test, nn_pred, average='weighted'),
                "ROC-AUC": roc_auc_score(y_test, nn_probs, multi_class='ovr', average='weighted')
        }

        # Display metrics
        print(metrics)
        conf_matrix = confusion_matrix(y_test, nn_pred)
        print("Confusion Matrix:")
        print(conf_matrix)

neural_network_regularizada(df, sex="all")
neural_network_regularizada(df, sex="F")
neural_network_regularizada(df, sex="M")

{'Accuracy': 0.6056338028169014, 'F1 Score': np.float64(0.6064026435032961), 'Precision': np.float64(0.6081460550134663), 'Recall': np.float64(0.6056338028169014), 'ROC-AUC': np.float64(0.7611843790012803)}
Confusion Matrix:
[[15  6  2]
 [ 5 16  6]
 [ 2  7 12]]
{'Accuracy': 0.5833333333333334, 'F1 Score': np.float64(0.5448412698412698), 'Precision': np.float64(0.6049382716049383), 'Recall': np.float64(0.5833333333333334), 'ROC-AUC': np.float64(0.754476207062414)}
Confusion Matrix:
[[ 4  2  1]
 [ 4  4  7]
 [ 1  0 13]]
{'Accuracy': 0.6666666666666666, 'F1 Score': np.float64(0.6666666666666666), 'Precision': np.float64(0.6666666666666666), 'Recall': np.float64(0.6666666666666666), 'ROC-AUC': np.float64(0.8030092592592594)}
Confusion Matrix:
[[11  1  1]
 [ 1  7  4]
 [ 1  4  6]]


/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/envs/FAA/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  warnings.warn(


In [816]:
def SVC_model_free(df, sex="all"):
        """svc model sem regularização (dps ler se é regularizacao e isso o termo correto)"""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Create a Support Vector Classifier model
        svc_model = SVC(kernel='linear')  # You can change the kernel to 'rbf', 'poly', etc.

        # Train the model
        svc_model.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = svc_model.predict(X_test)

        # Evaluate the model
        print("Accuracy:", accuracy_score(y_test, y_pred))
        #print("Classification Report:\n", classification_report(y_test, y_pred))

SVC_model_free(df, sex="all")
SVC_model_free(df, sex="F")
SVC_model_free(df, sex="M")


Accuracy: 0.6338028169014085
Accuracy: 0.6388888888888888
Accuracy: 0.7222222222222222


In [817]:
def SVC_model_regularizado(df, sex="all"):
        """svc model COM regularização (dps ler se é regularizacao e isso o termo correto)"""
        if sex=="all":
                filtered_df = df
        elif sex=="F":
                filtered_df = df[df["Sex_F"] > 0.5]
        elif sex=="M":
                filtered_df = df[df["Sex_F"] < 0.5]
        else:
                raise ValueError("Invalid")

        X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
        y = filtered_df['FatigInd']

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Create a Support Vector Classifier model
        svc_model = SVC(kernel='linear')  # You can change the kernel to 'rbf', 'poly', etc.
        svc_model_high_C = SVC(C=1000, kernel='linear')  # High penalty (focuses on reducing training error)
        svc_model_low_C = SVC(C=0.01, kernel='linear')  # Low penalty (allows more errors for larger margin)

        svc_model_high_C.fit(X_train, y_train)
        svc_model_low_C.fit(X_train, y_train)

        # Predictions
        y_pred_high_C = svc_model_high_C.predict(X_test)
        y_pred_low_C = svc_model_low_C.predict(X_test)

        # Evaluate accuracy
        print("Accuracy with high C (10):", accuracy_score(y_test, y_pred_high_C))
        print("Accuracy with low C (0.01):", accuracy_score(y_test, y_pred_low_C))

SVC_model_regularizado(df, sex="all")
SVC_model_regularizado(df, sex="F")
SVC_model_regularizado(df, sex="M")


Accuracy with high C (10): 0.5774647887323944
Accuracy with low C (0.01): 0.5352112676056338
Accuracy with high C (10): 0.4444444444444444
Accuracy with low C (0.01): 0.4166666666666667
Accuracy with high C (10): 0.75
Accuracy with low C (0.01): 0.5277777777777778
